In [8]:
#Import our dependencies

import zipfile
%matplotlib inline 
import numpy as np 
import scipy as sp 
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
import pandas as pd 
import sklearn
import seaborn as sns

import os
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

In [9]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [11]:
# Establish our path
events = os.path.join("Resources", "events.csv")

In [12]:
#Creata our Dataframe
events_df = pd.read_csv(events)
events_df

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941003,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
941004,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,NaN,2,Sampdoria,Atalanta,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
941005,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,NaN,NaN,8.0,2.0,0,9.0,1.0,1,3.0,0
941006,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,NaN,1,Atalanta,Sampdoria,...,NaN,NaN,NaN,NaN,0,4.0,NaN,0,NaN,0


In [13]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941008 entries, 0 to 941007
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id_odsp        941008 non-null  object 
 1   id_event       941008 non-null  object 
 2   sort_order     941008 non-null  int64  
 3   time           941008 non-null  int64  
 4   text           941008 non-null  object 
 5   event_type     941008 non-null  int64  
 6   event_type2    214293 non-null  float64
 7   side           941008 non-null  int64  
 8   event_team     941008 non-null  object 
 9   opponent       941008 non-null  object 
 10  player         880009 non-null  object 
 11  player2        291310 non-null  object 
 12  player_in      51715 non-null   object 
 13  player_out     51738 non-null   object 
 14  shot_place     227458 non-null  float64
 15  shot_outcome   228497 non-null  float64
 16  is_goal        941008 non-null  int64  
 17  location       467066 non-nul

In [14]:
#Show data columns

events_df.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [16]:
# Find out the values of the "is goal column"
events_df["is_goal"].unique()

array([0, 1], dtype=int64)

In [17]:
#find out the count of "is goal golumn"
events_df["is_goal"].value_counts()

0    916562
1     24446
Name: is_goal, dtype: int64

In [18]:
# Reading the dictionary data to see what event type are 

dictionary_text = open('dictionary.txt')

In [19]:
dictionary_text.read()

'event_type\n0\tAnnouncement\n1\tAttempt\n2\tCorner\n3\tFoul\n4\tYellow card\n5\tSecond yellow card\n6\tRed card\n7\tSubstitution\n8\tFree kick won\n9\tOffside\n10\tHand ball\n11\tPenalty conceded\n\n\nevent_type2\n12\tKey Pass\n13\tFailed through ball\n14\tSending off\n15\tOwn goal\n\n\nside\n1\tHome\n2\tAway\n\n\nshot_place\n1\tBit too high\n2\tBlocked\n3\tBottom left corner\n4\tBottom right corner\n5\tCentre of the goal\n6\tHigh and wide\n7\tHits the bar\n8\tMisses to the left\n9\tMisses to the right\n10\tToo high\n11\tTop centre of the goal\n12\tTop left corner\n13\tTop right corner\n\n\nshot_outcome\n1\tOn target\n2\tOff target\n3\tBlocked\n4\tHit the bar\n\n\nlocation\n1\tAttacking half\n2\tDefensive half\n3\tCentre of the box\n4\tLeft wing\n5\tRight wing\n6\tDifficult angle and long range\n7\tDifficult angle on the left\n8\tDifficult angle on the right\n9\tLeft side of the box\n10\tLeft side of the six yard box\n11\tRight side of the box\n12\tRight side of the six yard box\n13\t

In [20]:
encoding = pd.read_table('dictionary.txt',  names=('num','events'))
encoding

,num,events
0,event_type,NaN
1,0,Announcement
2,1,Attempt
3,2,Corner
4,3,Foul
...,...,...
70,situation,NaN
71,1,Open play
72,2,Set piece
73,3,Corner


In [21]:
## checking the events column
events_df["event_type"].unique()

array([ 1,  2,  3,  8, 10,  7,  9,  4,  6, 11,  5], dtype=int64)

In [22]:
#find out the count of "event_type"
events_df["event_type"].value_counts()

8     237932
3     232925
1     229134
2      91204
7      51738
9      43476
4      39911
10     10730
11      2706
6       1152
5        100
Name: event_type, dtype: int64

In [23]:
# We need the event_type when is equal 1 since that is a shot attempt.
shot_attempts = events_df[events_df["event_type"]== 1]

# We check the filter
shot_attempts.head() 

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
11,UFot0hit/,UFot0hit12,12,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,13.0,2.0,0,15.0,1.0,1,1.0,0
13,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0
14,UFot0hit/,UFot0hit15,15,19,Attempt blocked. Mats Hummels (Borussia Dortmu...,1,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,2.0,3.0,0,15.0,1.0,0,1.0,0
17,UFot0hit/,UFot0hit18,18,20,Attempt blocked. Tomas Rincon (Hamburg) right ...,1,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,2.0,3.0,0,15.0,1.0,0,1.0,0


In [24]:
### We'll check is the number is the same in both databases
shot_attempts["event_type"].value_counts()

1    229134
Name: event_type, dtype: int64

In [25]:
shot_attempts["player"].unique()

array(['mladen petric', 'shinji kagawa', 'kevin grosskreutz', ...,
       'alhassan wakaso', 'roman zozulia', 'alessandro bastoni'],
      dtype=object)

In [26]:
# Since is goal is an integer (meaning 1 is goal), we can sum the "is goal" column to get the top 50 scorers
filter_players= events_df.groupby(['player'])['is_goal'].sum().reset_index()
sorted_players = filter_players.sort_values('is_goal', ascending=False)
sorted_players.head(50)

,player,is_goal
3518,lionel messi,205
1215,cristiano ronaldo,198
6109,zlatan ibrahimovic,153
5074,robert lewandowski,124
1629,edinson cavani,121
2275,gonzalo higuain,118
4869,pierreemerick aubameyang,100
3660,luis suarez,96
1509,diego costa,93
312,alexandre lacazette,88


In [34]:
# Checking one player
neymar = (shot_attempts.player == 'neymar')
totalshots_neymar = shot_attempts.player[neymar].count()
totalshots_neymar


316

In [35]:
#check the ratio for the efectivness of Neymar
totalgoals_neymar = shot_attempts.player[neymar][shot_attempts.is_goal == 1].count()
effectivness_neymar = (totalgoals_neymar/totalshots_neymar) * 100

print('Neymar scored', totalgoals_neymar, 'goals with a total of', totalshots_neymar, 'for an efectivness of', effectivness_neymar.round(2), '%' )

Neymar scored 56 goals with a total of 316 for an efectivness of 17.72 %


In [36]:
# According to our dictionary database there are some shot locations that are not recorded (number 19)
print(shot_attempts.player[shot_attempts.location == 19].count()) 

0


In [37]:
# Let's see the percentage. 
print(shot_attempts.event_type[shot_attempts.location == 19].count()/shot_attempts["event_type"].value_counts()* 100) 

1    0.0
Name: event_type, dtype: float64


In [38]:
#Since is less than 1% we are going to drop those values, even though it may affect our results, it will have a minimal effect

shot_attempts = shot_attempts[shot_attempts.location != 19]


In [39]:
#cHECK NULL VALUIES
shot_attempts.isna().any()

id_odsp          False
id_event         False
sort_order       False
time             False
text             False
event_type       False
event_type2       True
side             False
event_team       False
opponent         False
player           False
player2           True
player_in         True
player_out        True
shot_place        True
shot_outcome     False
is_goal          False
location         False
bodypart         False
assist_method    False
situation        False
fast_break       False
dtype: bool

In [40]:
# count of null values in each column. 
print(shot_attempts.isnull().sum())



id_odsp               0
id_event              0
sort_order            0
time                  0
text                  0
event_type            0
event_type2       59825
side                  0
event_team            0
opponent              0
player                0
player2           59886
player_in        227684
player_out       227684
shot_place         1008
shot_outcome          0
is_goal               0
location              0
bodypart              0
assist_method         0
situation             0
fast_break            0
dtype: int64


In [41]:
#We can see that the values we are going to use in the machine learning are not null

In [42]:

pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [43]:
#Check the columns
shot_attempts.columns


Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [29]:
# Now to Finish let's export our database to a new spreadsheet to uae for visualizations
shot_attempts.to_excel(r'C:\Users\gotar\Desktop\Scouting_Player_Report\Resources\clean_data.xlsx', index = False, header=True)